In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min width:70px;}
div#toc-wrapper {padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe {font-size:12px;}
</style>
"""))

<b><font size='6' color='red'>ch14 웹데이터 수집 정적</font></b>
# 1절. Beautiful Soup과 parser
'pip install bs4' 아나콘다를 설치하면 자동 설치되는 7500개의 패키지 중 하나
- 공식 documentation 사이트 : https://tedboy.github.io/bs4_doc/

In [2]:
# 내 local PC의 파일을 url처럼 접근
import requests # http 요청 처리하는 lib
from requests_file import FileAdapter # file:// 프로토콜 다루기 위함

In [3]:
s = requests.Session() # HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file:// 경로를 들어오면 이 요청은 C:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response

<Response [200]>

In [4]:
if response: # reponse 200일 경우만 if문 통과
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다.')

잘 접근하였습니다.


In [ ]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406 : method가 이상함(get, post 오류)

In [5]:
response.content #비이너리 형식의 html 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [6]:
response.content.decode('utf-8') # html의 meta charset과 동일하게

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [7]:
response.text # HTML파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
# html 파싱
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, #response.text
                    'html.parser')

In [14]:
# soup.select_one('선택자') : 해당 선택자가 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1 태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성들 :', el.attrs)
print('el의 id 속성 :', el.attrs['id']) # el.attrs은 딕셔너리
print('el의 id 속성 :', el.get('id'))
print('el의 href 속성 :', el.get('href'))
print('el의 태그이름 :', el.name)

el : <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id 속성 : text
el의 id 속성 : text
el의 href 속성 : None
el의 태그이름 : h1


In [20]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1') # h1 태그를 리스트로
print('els', els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.text for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

els [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : ['Hello, CSS', 'Hi, CSS']
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [25]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css'))
print('find :', soup.find('h1', {'class' : 'css'})) # soup.select_one('h1.css')
print('find :', soup.find('h1', class_='css'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [27]:
# soup.findall(태그, 속성) : soup.select('선택자')와 유사
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [36]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], class_='css'))
print('find_all :', soup.find_all(['h1'], class_='css') + soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [40]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a.css'))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
## 2.1 Beautiful Soup을 이용한 html 웹 데이터수집
### 1) 환율 정보 가져이고 (네이버 증권 -> 시장지표)
- https://finance.naver.com/marketindex/

In [42]:
# 크롤링 허용범위는 사이트마다 ~/robots.txtx에서 확인
    # Allow : / - 사이트의 모든 경로(/)에 대한 크롤링 허용
# 방법 1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [48]:
# 방법 2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
#response.status
#response.read().decode('cp949')
soup = BeautifulSoup(response, 'html.parser')

In [55]:
# 가격 : div.head_info>span.value (find함수)
prices = []
#find 사용시
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(','))))
prices

[1454.5,
 945.19,
 1681.33,
 204.2,
 153.42,
 1.1564,
 1.3161,
 99.47,
 59.75,
 1698.27,
 4009.8,
 189125.23]

In [60]:
# 가격 : div.head_info>span.value (select함수)
prices = soup.select('div.head_info>span.value')
[float(p.text.replace(',','')) for p in prices]

[1454.5,
 945.19,
 1681.33,
 204.2,
 153.42,
 1.1564,
 1.3161,
 99.47,
 59.75,
 1698.27,
 4009.8,
 189125.23]

In [65]:
# 타이틀 : h3.h_lst>span.blind
titles = soup.select('h3.h_lst>span.blind')
[title.text for title in titles]

['미국 USD',
 '일본 JPY(100엔)',
 '유럽연합 EUR',
 '중국 CNY',
 '달러/일본 엔',
 '유로/달러',
 '영국 파운드/달러',
 '달러인덱스',
 'WTI',
 '휘발유',
 '국제 금',
 '국내 금']

In [67]:
# 유닛 : div.head_info>span>span.blind
units = soup.select('div.head_info>span>span.blind')
units = [unit.text for unit in units]
units.insert(7, '') # 7번째에 추가
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [69]:
# 상승/하강 : div.head_info>span.blind
status = soup.select('div.head_info>span.blind')
[s.text for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '상승', '하락', '상승', '상승', '상승', '상승']

In [70]:
len(titles), len(prices), len(units), len(status)

(12, 12, 12, 12)

In [71]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text, prices[idx].text, units[idx], status[idx].text))

미국 USD : 1,454.50원 - 하락
일본 JPY(100엔) : 945.19원 - 하락
유럽연합 EUR : 1,681.33원 - 하락
중국 CNY : 204.20원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1698.27원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 189125.23원 - 상승


In [72]:
for title, price, unit, s in zip(titles, prices, units, status):
    print("{} : {}{} - {}".format(title.text, price.text, unit, s.text))

미국 USD : 1,454.50원 - 하락
일본 JPY(100엔) : 945.19원 - 하락
유럽연합 EUR : 1,681.33원 - 하락
중국 CNY : 204.20원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1698.27원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 189125.23원 - 상승


### 2) 이번주 로또번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin (google에 로또번호 당첨번호 검색)
```
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1, 5, 7, 26, 28, 43]
    보너스 30
```

In [76]:
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [77]:
# num(회차) : div.win_result>h4>strong
num = soup.select_one('div.win_result>h4>strong')
num.text

'1197회'

In [79]:
# date : div.win_result>p.desc
date = soup.select_one('div.win_result>p.desc')
date.text

'(2025년 11월 08일 추첨)'

In [84]:
# ball_num(당첨공번호와 보너스) : div.num>span.ball_645
ball_nums = soup.select('div.num>p>span.ball_645')
ball_nums

[<span class="ball_645 lrg ball1">1</span>,
 <span class="ball_645 lrg ball1">5</span>,
 <span class="ball_645 lrg ball1">7</span>,
 <span class="ball_645 lrg ball3">26</span>,
 <span class="ball_645 lrg ball3">28</span>,
 <span class="ball_645 lrg ball5">43</span>,
 <span class="ball_645 lrg ball3">30</span>]

In [88]:
#결과
print(num.text + date.text)
print('당첨번호', [int(ball_num.text) for ball_num in ball_nums[:-1]])
print('보너스 :', int(ball_nums[-1].text))

1197회(2025년 11월 08일 추첨)
당첨번호 [1, 5, 7, 26, 28, 43]
보너스 : 30


### 3) 다음 검색 리스트
- https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=검색워드

```
no title                 href
0  "한풀꺽인 비트코인.."   링크주소
```

In [105]:
# 방법 1
import requests
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=' + word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

items_find_list = [] #검색할 결과를 담을 리스트 (딕셔너리 list)
# title : div.item-title>strong>a
item_el = soup.select('div.item-title>strong>a');
for idx, item in  enumerate(item_el):
    #print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx, 'title':item.text, 'link':item.attrs.get('href')})

import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
9,9,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340


In [104]:
#방법 2 : 불가
# 이유: 1. url에 한글이 있음(한글을 인코딩해야한다) : 해결가능 2. robots.txt를 보면 user-agent만 allow고 나머지는 disallow이다 : 해결불가
from urllib.request import urlopen
from urllib.parse import quote
word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 인코딩해야한다
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=' + word
print(url)
response = urlopen(url)
soup = BeautifulSoup(response, 'html.parser')

items_find_list = [] #검색할 결과를 담을 리스트 (딕셔너리 list)
# title : div.item-title>strong>a
item_el = soup.select('div.item-title>strong>a');
for idx, item in  enumerate(item_el):
    #print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx, 'title':item.text, 'link':item.attrs.get('href')})

import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


""


In [110]:
items_find_list = [] #검색할 결과를 담을 리스트 (딕셔너리 list)
# title : div.item-title>strong>a
item_el = soup.select('div.item-title>strong>a');
for idx, item in  enumerate(item_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크']).set_index('번호')

,제목,링크
번호,,
0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519


In [119]:
items_find_list = [] #검색한 결과를 담을 리스트
# div.item-title > strong > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    items_find_list.append([idx, a.text, a.attrs.get('href')])
pd.DataFrame(items_find_list, columns=['번호', '제목', '링크']).set_index('번호')

,제목,링크
번호,,
0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
2,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
3,"트럼프 돈 푼다...비트코인, 관세 배당금에 반등 [매일코인]",http://v.daum.net/v/20251110112101913
4,"비트코인, 트럼프 ‘2000달러 지급’ 발언에 10만4000달러선 [크립토브리핑]",http://v.daum.net/v/20251110103033108
5,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
6,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
7,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
8,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519


In [139]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지 수만큼 조회)
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_daum_news_list(keywords, page=1):
    news_lists = []
    for idx, keyword in enumerate(keywords):
        news_list = []
        print(f'==={idx+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
        for i in range(1,page+1):
            print(i, keyword)
            #url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={}&p={}'.format(keyword, i)
            #response = requests.get(url)
            url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
            params={'q':keyword, 'p':i}
            response = requests.get(url, params=params)
            soup = BeautifulSoup(response.text, 'html.parser')
            news_els = soup.select('div.item-title>strong>a')
            for idx, item in enumerate(news_els):
                #print((i-1)*10+idx, item.text, item.attrs.get('href'))
                news_list.append({'번호':(i-1)*10+idx, '제목':item.text, '링크':item.attrs.get('href')})
        news_lists.append(pd.DataFrame(news_list))
    return news_lists

keywords=['야구','해외']
result = collect_daum_news_list(keywords, 3)

===1번째 검색어 야구 검색 결과 수집 중입니다===
1 야구
2 야구
3 야구
===2번째 검색어 해외 검색 결과 수집 중입니다===
1 해외
2 해외
3 해외


In [140]:
result[0].to_csv(f'data/ch14_{keywords[0]}.csv', index=False)
result[1].to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것 처럼 보이기

In [146]:
from urllib.request import urlopen, Request
from urllib.parse import quote

word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 인코딩해야한다
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=' + word
print(url)
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


In [148]:
from urllib.request import urlopen, Request
from urllib.parse import quote

word = quote('비트코인') # url에 한글이 포함된 경우 한글을 quote()로 인코딩해야한다
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=' + word
print(url)
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url)
request.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')
response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
#soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


- https://www.melon.com/chart/index.htm <br>
    https://www.melon.com/robots.txt 에서 User-Agent에 봇이 지정

In [154]:
# 방법 1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup

<Response [406]>

In [158]:
#방법 2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
#soup

In [163]:
#방법 1 : User-Agent 추가
import requests
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
melonpage
soup = BeautifulSoup(melonpage.text, 'html.parser')
#soup

In [171]:
#melon에서 노래 정보 추출 : 
#     순위 : td div.wrap.t_center > span.rank
#     노래제목 : div.ellipsis.rank01 > span
#     가수 : div.ellipsis.rank02
ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') #title.text.strip()
singers = soup.select('div.ellipsis.rank02')
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Drowning - WOODZWOODZ
5위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | FAMOUS - ALLDAY PROJECTALLDAY
12위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
16위 | Rich Man - aespaaespa
17위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
18위 | 한번 더 이별 - 이창섭이창섭
19위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위 | 너에게 닿기를 - 10CM10CM
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 천상연 - 이창섭이창섭
28위 | 나는 반딧불 - 황가람황가람
29위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AK

### 5) 네이버 지식인으로 검색 (openAPI 사용x)
- 특정 키워드를 특정 페이지 수만큼

In [6]:
# 방법 1
from requests import get
from bs4 import BeautifulSoup

keyword = '챗지피티'
url = 'https://kin.naver.com/search/list.naver?query='+keyword
print(url)
response = get(url)
print(response.status_code)

soup = BeautifulSoup(response.text, 'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=챗지피티
200


In [9]:
# 방법 2
from urllib.request import urlopen
from urllib.parse import quote
keyword = quote('챗지피티')
url = 'https://kin.naver.com/search/list.naver?query='+keyword

response = urlopen(url)
print(response.status)
soup = BeautifulSoup(response, 'html.parser')
soup

200



<!DOCTYPE html>

<html lang="ko">
<head>
<meta contents="always" name="referrer"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="챗지피티의 지식iN Q&amp;A 검색결과입니다. 궁금증을 해결하지 못했다면 지식iN '질문하기'를 해보세요." name="description">
<meta content="width=1024" name="viewport"/>
<meta content="none" name="msapplication-config">
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/components.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/static.kin/static/pc/20251029142034/css/min/other.css" rel="stylesheet" type="text/css"/>
<script>
	
	var pcDomain = "kin.naver.com";
	var pcDomainWithProtocol = "https://kin.naver.com";
	var pcKinServiceProtocol = "https";

	var mobileDomain = "m.kin.naver.com";
	var mobileDomainWithProtocol = "https://m.kin.naver.com";
	var mobileKinServiceProtocol = "https";

	var uplo

In [17]:
# 페이징 포함
from requests import get
from bs4 import BeautifulSoup
import pandas as pd

keyword = '챗지피티'
pages = 3
item_list = [] # 크롤링한 데이터를 담을 list (title, link)
for page in range(1, pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('dt > a')
    for item in items:
        title = item.text
        link = item.attrs.get('href')
        item_list.append({'title':title, 'link':link})
df = pd.DataFrame(item_list)
print(df.shape)
print(df.loc[29, 'link'])
df.head()

(30, 2)
https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1060101&docId=486781346&qb=7LGX7KeA7ZS87Yuw&enc=utf8


,title,link
0,챗지피티 관련 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...
1,챗지피티 자동 결제 안 됨,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,챗지피티 쉐어,https://kin.naver.com/qna/detail.naver?d1id=1&...
3,챗지피티 전송 오류,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,Ai 챗지피티 답변,https://kin.naver.com/qna/detail.naver?d1id=1&...


## 2.2 openAPI 사용 : json 웹데이터 수집
### 1) 네이버 지식인으로 검색 (OpenAPI)
- 네이버 개발자센터에서 어플리케이션(이름/검색/WEB 설정http://localhost)
- .env파일에 CLIENT_ID/CLIENT_SECRET
- 환경변수를 읽기 위해서 `pip install dotenv`
- 특정 keyword 지식검색(데이터수 30개)

In [18]:
#%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [31]:
# 환경변수 읽어오기
from dotenv import load_dotenv
import os
load_dotenv(
        #dotenv_path='env'
    ) # 현 소스와 같은 폴더내의 .env를 메모리에 load
# print(os.getenv('CLIENT_ID'))
# print(os.getenv('CLIENT_SECRET'))

True

In [43]:
# 방법2
import os
import urllib.request
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
encText = urllib.parse.quote("챗지피티")
url = "https://openapi.naver.com/v1/search/kin?query=" + encText # JSON 결과
# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)
headers = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}
request = urllib.request.Request(url, headers=headers)
response = urllib.request.urlopen(request)
rescode = response.status
if(rescode==200):
    response_body = response.read()
    print(json.loads(response_body.decode('utf-8')))
else:
    print("Error Code:" + rescode)

{'lastBuildDate': 'Tue, 11 Nov 2025 11:47:23 +0900', 'total': 33779, 'start': 1, 'display': 10, 'items': [{'title': '<b>챗지피티</b> 관련 질문', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=102020104&docId=489639600&qb=7LGX7KeA7ZS87Yuw&enc=utf8', 'description': '<b>챗지피티</b> 계정 삭제시 <b>챗지피티</b>에서 활동했던 기록(대화, 사진 등)들 다 서버에서 삭제되나요? 누구도 볼 수 없나요? 안녕하세요? 지식파트너 가비아입니다. 챗GPT 계정을 삭제하면 OpenAI 서버에 저장된 모든 데이터가 30일 이내에... '}, {'title': '<b>챗지피티</b> 자동 결제 안 됨', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1070401&docId=487902448&qb=7LGX7KeA7ZS87Yuw&enc=utf8', 'description': '7월 7일에 <b>챗지피티</b> 프로 결제해서 한달 열심히 썼습니다. 이번 달에도 7일쯤... <b>챗지피티</b> 사이트에서 결제하면 자동 결제가 안 되는 건가요?? 미결제 관련... 혹시 <b>챗지피티</b> 계정 공유로 월 4달러~6달러 수준으로 절약하는 방법 궁금하시면... '}, {'title': '<b>챗지피티</b> 쉐어', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=104&docId=489500740&qb=7LGX7KeA7ZS87Yuw&enc=utf8', 'description': '... 에서 <b>챗지피티</b> 쉐어를 하려고 하는데 <b>챗지피티</b>는 계정공유가 어떤식으로 진행되는건가요? 다른 사람들이 내가 사용한 <b>챗지피티</b>

In [58]:
#방법 1
import requests
import pandas as pd
import json
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
query = '챗지피티'
headers = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}
# url = f"https://openapi.naver.com/v1/search/kin?query={query}&display=30"
# response = requests.get(url, headers=headers)
url = "https://openapi.naver.com/v1/search/kin"
params = {'query':query, 'display':30}
response = requests.get(url, params=params, headers=headers)
# items = json.loads(response.text)['items']
items = response.json()['items']
items_list = []
for item in items:
    title = item['title'].replace('<b>','').replace('</b>','')
    link = item.get('link')
    description = item.get('description').replace('<b>','').replace('</b>','')
    items_list.append([title, link, description])
pd.DataFrame(item_list, columns=['title', 'link', 'description']).head()

,title,link,description
0,챗지피티 관련 질문,https://kin.naver.com/qna/detail.naver?d1id=1&...,NaN
1,챗지피티 자동 결제 안 됨,https://kin.naver.com/qna/detail.naver?d1id=1&...,NaN
2,챗지피티 쉐어,https://kin.naver.com/qna/detail.naver?d1id=1&...,NaN
3,챗지피티 전송 오류,https://kin.naver.com/qna/detail.naver?d1id=1&...,NaN
4,Ai 챗지피티 답변,https://kin.naver.com/qna/detail.naver?d1id=1&...,NaN


### quiz) 네이버 open API를 이용해서 청바지 이미지 100건의 데이터를 ch14_청바지.csv로 출력
```
제목,링크,썸네일,sizeheight,sizewidth
```

In [74]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
headers = {
    "X-Naver-Client-Id":client_id,
    "X-Naver-Client-Secret":client_secret
}
url = 'https://openapi.naver.com/v1/search/image'
item_list = []
params = {'query':'청바지', 'display':100}
response = requests.get(url, params=params, headers=headers)
items = response.json()['items']
for item in items:
    title = item['title']
    link = item['link']
    thumbnail = item['thumbnail']
    sizeheight = item['sizeheight']
    sizewidth = item['sizewidth']
    item_list.append({
        '제목':title,
        '링크':link,
        '썸네일':thumbnail,
        '높이':sizeheight,
        '폭':sizewidth
    })

df = pd.DataFrame(item_list)
df.to_csv('data/ch14_청바지.csv', index=False)
df.head()

,제목,링크,썸네일,높이,폭
0,여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...,https://shop-phinf.pstatic.net/20251015_281/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,-30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠,http://shopping.phinf.naver.net/main_5666884/5...,https://search.pstatic.net/common/?type=b150&s...,600,600
2,키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠,http://shop1.phinf.naver.net/20240320_292/1710...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
3,작은 자른 슬릿 높은 허리 청바지 2022 봄 여름 얇은 스트레이트 느슨한 모든 일...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800


In [80]:
def save_image(attr, idx, link, query):
    'link의 이미지를 image/attr_idx_query, 확장자로 local에 저장'
    file_extension = link.split('.')[-1] #확장자
    quote_index = file_extension.find('?')
    if quote_index != -1:
        file_extension = file_extension[:quote_index]
    img = requests.get(link).content #이미지이기에 text가 안됨
    with open(f'image/{attr}_{idx+1}_{query}.{file_extension}','wb') as f:
        f.write(img)
save_image('메인', 0, df.loc[0,'링크'], '청바지')

In [82]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os

def get_image_list_save_image(query):
    load_dotenv()
    client_id = os.getenv('CLIENT_ID')
    client_secret = os.getenv('CLIENT_SECRET')
    headers = {
        "X-Naver-Client-Id":client_id,
        "X-Naver-Client-Secret":client_secret
    }
    url = 'https://openapi.naver.com/v1/search/image'
    item_list = [] #정보가 담길 리스트
    params = {'query':query, 'display':100}
    response = requests.get(url, params=params, headers=headers)
    items = response.json()['items']
    for idx, item in enumerate(items):
        title = item['title']
        link = item['link']
        thumbnail = item['thumbnail']
        sizeheight = item['sizeheight']
        sizewidth = item['sizewidth']
        item_list.append({
            '제목':title,
            '링크':link,
            '썸네일':thumbnail,
            '높이':sizeheight,
            '폭':sizewidth
        })
        #이미지 파일 저장
        save_image('메인',idx, link, query)
        save_image('썸네일', idx, link, query)
        if (idx%20 == 0) & (idx!=0):
            print(f'= = = {idx}% 진행완료 = = = ')
    print('= = = 완료 = = =')
    return pd.DataFrame(item_list)
df = get_image_list_save_image('청바지')
df.to_csv('image/ch14_청바지.csv', index=False)
df.head()

= = = 20% 진행완료 = = = 
= = = 40% 진행완료 = = = 
= = = 60% 진행완료 = = = 
= = = 80% 진행완료 = = = 
= = = 완료 = = =


,제목,링크,썸네일,높이,폭
0,여성 하이웨이스트 연청 와이드 청바지 통바지 편안함 스타일리시 캐주얼 데님 : 믿음...,https://shop-phinf.pstatic.net/20251015_281/17...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
1,-30 여자 청바지 여성 데님 밴딩 일자 통 와이드 팬츠,http://shopping.phinf.naver.net/main_5666884/5...,https://search.pstatic.net/common/?type=b150&s...,600,600
2,키작녀청바지 세미와이드청바지 진청바지 : 미니팬츠,http://shop1.phinf.naver.net/20240320_292/1710...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
3,작은 자른 슬릿 높은 허리 청바지 2022 봄 여름 얇은 스트레이트 느슨한 모든 일...,https://thumbnail.coupangcdn.com/thumbnails/re...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,JEEP spirit 남성 청바지 스판 남자 캐쥬얼 팬츠 지프 : 우수한 편의점,https://shop-phinf.pstatic.net/20251014_131/17...,https://search.pstatic.net/common/?type=b150&s...,800,800


## 2.3 XML 웹데이터 수집
- RSS / open API을 통한 XML 웹 데이터 수집

### 1) 전국 날씨 RSS를 BeautfiulSoup을 이용한 크롤링
- 구글에 "기상청 RSS"
    - https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml

In [96]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

items_list = []
url = 'https://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20251106.xml'

#방법 1
target = requests.get(url)
soup = BeautifulSoup(target.text, 'xml') #pip install lxml 필요없음

#방법 2
# target = urlopen(url)
# soup = BeautifulSoup(target, 'xml')

locals_ = soup.select('local_ta')
len(locals_)
for local in locals_:
    local_name = local.select_one('local_ta_name').text
    week1_normalYear = local.select_one('week1_local_ta_normalYear').text
    week1_similarRange = local.select_one('week1_local_ta_similarRange').text
    week1_minVal = local.select_one('week1_local_ta_minVal').text
    week1_similarVal = local.select_one('week1_local_ta_similarVal').text
    week1_maxVal = local.select_one('week1_local_ta_maxVal').text
    
    
    items_list.append({
        '지역':local_name,
        '1주평년기온':week1_normalYear,
        '1주기온범위':week1_similarRange,
        '1주낮을확률':week1_minVal,
        '1주비슷할확률':week1_similarVal,
        '1주높을확률':week1_maxVal
    })
df = pd.DataFrame(items_list)
df['1주평년기온'] = df['1주평년기온'].astype(np.float64)
df['1주낮을확률'] = df['1주낮을확률'].astype(np.int16)
df['1주비슷할확률'] = df['1주비슷할확률'].astype(np.int16)
df['1주높을확률'] = df['1주높을확률'].astype(np.int16)
df.head()

,지역,1주평년기온,1주기온범위,1주낮을확률,1주비슷할확률,1주높을확률
0,"전국(제주도,북한제외)",5.9,5.0~6.8,20,40,40
1,서울ㆍ인천ㆍ경기도,4.8,3.7~5.9,20,40,40
2,강원도 영서,2.6,1.5~3.7,20,40,40
3,강원도 영동,5.8,4.9~6.7,20,40,40
4,대전ㆍ세종ㆍ충청남도,5.4,4.5~6.3,20,40,40


### 2) XML응답하는 openAPI활용
- data.go.kr에서
    - 서울특별시_노선정보조회서비스(버스ID, 정류장목록과정류장ID)
    - 서울특별시_버스위치정보조회서비스(실시간 버스 위치 목록)

In [2]:
# STEP 1 버스번호와 버스 id 받아오기
# 서울특별시_노선정보조회 서비스 - 3번 기능(getBusRouteList) 이용
from dotenv import load_dotenv
import os
load_dotenv()
# print(os.getenv('GOVDATAKEY'))

True

In [3]:
import requests
from bs4 import BeautifulSoup
# urlretrieve(url, 저장경로) : url의 파일을 저장경로에 저장
from urllib.request import urlretrieve
from urllib.parse import quote

busNum = quote('구로09')
# busNum = '273'
key = os.getenv('GOVDATAKEY')
url =f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
savefilename1 = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename1) # xml파일(url)을 savefilename으로 저장
with open(savefilename1, encoding='utf-8') as f:
    xml = f.read();
soup = BeautifulSoup(xml, 'xml')

In [4]:
import requests
from urllib.parse import quote

busNum = '162'
key = os.getenv('GOVDATAKEY')
url =f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')

for item in soup.select('itemList'):
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum:
        busRouteId = item.select_one('busRouteId').text
        break;
print('busRouteId =', busRouteId)

busRouteId = 100100034


In [ ]:
# step2. 버스 id로 버스정류장목록받아오기(정류장명, id, 경도, 위도)
# 서울특별시_노선정보조회_서비스 - 4번기능(getSTationByRoutelist)

In [5]:
import pandas as pd
url = f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')
item_list = soup.select('itemList')
print(f'{busNum}의 정류장 갯수 : {len(item_list)}')
bus_stations = []
for item in item_list:
    stationNm = item.select_one('stationNm').text #정류소이름
    station = item.select_one('station').text # 정류장 id
    gpsX = item.select_one('gpsX').text # 정류장 경도
    gpsY = item.select_one('gpsY').text # 정류장 위도
    bus_stations.append([stationNm, station, gpsX, gpsY])
df_stations = pd.DataFrame(bus_stations, columns=['정류소명', 'id', '경도', '위도'])
df_stations.head()

162의 정류장 갯수 : 77


,정류소명,id,경도,위도
0,정릉산장아파트,107000071,127.003343,37.616712
1,정릉4동주민센터.경국사,107000073,127.006345,37.613529
2,북한산보국문역2번출구,107000518,127.0079858233,37.612293899
3,성북청수도서관.정릉4동성당,107000075,127.0084193769,37.6115696748
4,정릉시장입구,107000077,127.0098212542,37.6084653256


In [32]:
# step3. 버스id로 실시간위치정보를 받아오기 (차량번호, 혼잡도, 최종정류장id, 다음정류장id, 도착소요시간)
#서울특별시_버스위치정보조회_서비스 - 2번기능(getBusPosByRtidList)
url = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
#print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, 'xml')
item_list = soup.select('itemList')
print('운행중인 버스는 {}대입니다'.format(len(item_list)))
bus_position = []
congetion_list = {'0':'없음', '3':'여유', '4':'보통', '5':'혼잡', '6':'매우혼잡'}
for item in item_list:
    plainNo = item.select_one('plainNo').text #차량번호
    congetion = congetion_list.get(item.select_one('congetion').text) # 혼잡도
    # (0 : 없음, 3 : 여유, 4 : 보통, 5 : 혼잡, 6 : 매우혼잡)
    gpsX = item.select_one('gpsX').text
    gpsY = item.select_one('gpsY').text
    lastStnId = item.select_one('lastStnId').text
    nextStId = item.select_one('nextStId').text
    nextStTm = item.select_one('nextStTm').text #다음 정류소까지 소요시간
    bus_position.append({'차량번호':plainNo, '경도':gpsX, '위도':gpsY, '혼잡도':congetion, '최종정류장id':lastStnId, '다음정류장id':nextStId, '도착소요시간(초)':nextStTm})
df_position = pd.DataFrame(bus_position)
df_position

운행중인 버스는 22대입니다


,차량번호,경도,위도,혼잡도,최종정류장id,다음정류장id,도착소요시간(초)
0,서울74사2218,127.006181,37.613663,여유,107000073,107000079,309
1,서울70사6558,127.014931,37.597029,보통,107000172,101000042,1527
2,서울74사1638,127.001744,37.586243,여유,100000005,101000042,1038
3,서울74사1536,126.990981,37.577584,여유,100000002,101000042,835
4,서울74사2244,126.98274,37.569391,보통,100000106,101000042,336
5,서울74사3381,126.972637,37.554678,여유,101000004,102000018,396
6,서울74사2201,126.966929,37.538614,여유,102000047,118000061,450
7,서울74사1967,126.962035,37.535692,여유,102000050,118000061,314
8,서울70사6556,126.926819,37.519049,여유,118000068,118000070,954
9,서울74사1537,126.918321,37.528008,여유,118000071,118000070,837


In [33]:
df_stations.loc[df_stations['id'] == '107000073', '정류소명'].iloc[0]

'정릉4동주민센터.경국사'

In [47]:
def station_name(row):
    row['이전정류장명'] = df_stations.loc[df_stations['id'] == row['최종정류장id'], '정류소명'].iloc[0]
    row['다음정류장명'] = df_stations.loc[df_stations['id'] == row['다음정류장id'], '정류소명'].iloc[0]
    minute = int(row['도착소요시간(초)'])//60  # //:몫연산자
    sec    = int(row['도착소요시간(초)']) % 60 
    row['도착소요시간'] = f'{minute}분 {sec}초'
    return row
station_name(df_position.loc[2])

차량번호             서울74사1638
경도              127.001744
위도               37.586243
혼잡도                     여유
최종정류장id          100000005
다음정류장id          101000042
도착소요시간(초)             1038
이전정류장명       혜화동로터리.여운형활동터
다음정류장명         해운센터.롯데영플라자
도착소요시간             17분 18초
Name: 2, dtype: object

In [50]:
df_position = df_position.apply(station_name, axis=1)

,차량번호,경도,위도,혼잡도,이전정류장명,다음정류장명,도착소요시간
0,서울74사2218,127.006181,37.613663,여유,정릉4동주민센터.경국사,정릉우체국앞,5분 9초
1,서울70사6558,127.014931,37.597029,보통,돈암2동주민센터입구.흥천사,해운센터.롯데영플라자,25분 27초
2,서울74사1638,127.001744,37.586243,여유,혜화동로터리.여운형활동터,해운센터.롯데영플라자,17분 18초
3,서울74사1536,126.990981,37.577584,여유,창경궁.서울대학교병원,해운센터.롯데영플라자,13분 55초
4,서울74사2244,126.98274,37.569391,보통,조계사.종로경찰서,해운센터.롯데영플라자,5분 36초
5,서울74사3381,126.972637,37.554678,여유,서울역버스환승센터,남영역.민주화운동기념관,6분 36초
6,서울74사2201,126.966929,37.538614,여유,용산꿈나무종합타운.원효로우체국,시범아파트.대교아파트,7분 30초
7,서울74사1967,126.962035,37.535692,여유,원효로2가,시범아파트.대교아파트,5분 14초
8,서울70사6556,126.926819,37.519049,여유,샛강역1번출구.여의도자이,여의도역6번출구,15분 54초
9,서울74사1537,126.918321,37.528008,여유,여의도역5번출구,여의도역6번출구,13분 57초


In [51]:
df_position.drop(['최종정류장id', '다음정류장id', '도착소요시간(초)'], axis=1, inplace=True)
df_position

,차량번호,경도,위도,혼잡도,이전정류장명,다음정류장명,도착소요시간
0,서울74사2218,127.006181,37.613663,여유,정릉4동주민센터.경국사,정릉우체국앞,5분 9초
1,서울70사6558,127.014931,37.597029,보통,돈암2동주민센터입구.흥천사,해운센터.롯데영플라자,25분 27초
2,서울74사1638,127.001744,37.586243,여유,혜화동로터리.여운형활동터,해운센터.롯데영플라자,17분 18초
3,서울74사1536,126.990981,37.577584,여유,창경궁.서울대학교병원,해운센터.롯데영플라자,13분 55초
4,서울74사2244,126.98274,37.569391,보통,조계사.종로경찰서,해운센터.롯데영플라자,5분 36초
5,서울74사3381,126.972637,37.554678,여유,서울역버스환승센터,남영역.민주화운동기념관,6분 36초
6,서울74사2201,126.966929,37.538614,여유,용산꿈나무종합타운.원효로우체국,시범아파트.대교아파트,7분 30초
7,서울74사1967,126.962035,37.535692,여유,원효로2가,시범아파트.대교아파트,5분 14초
8,서울70사6556,126.926819,37.519049,여유,샛강역1번출구.여의도자이,여의도역6번출구,15분 54초
9,서울74사1537,126.918321,37.528008,여유,여의도역5번출구,여의도역6번출구,13분 57초


# 3절 연습문제(Quiz)
- yes24의 베스트셀러 정보를 제공하는 사이트에서 베스트셀러 정보를 수집해서 파일에 저장하세요 : http://www.yes24.com/24/category/bestseller
- ch14_yes.csv(또는 txt)
 - 순위, 책이름, 저자 및 출판사, 가격

In [88]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

page=2
url = 'http://www.yes24.com/24/category/bestseller='
b_list = []
for i in range(1,page+1):
    params = {'pageNumber':i, 'pageSize':24}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    books_list = soup.select('div.itemUnit')
    for book in books_list:
        rank = int(book.select_one('em.ico.rank').text)
        name = book.select_one('div.info_name > a.gd_name').text
        auth = book.select_one('span.info_auth').text.split('정보 더 보기/감추기')[0].strip()
        pub = book.select_one('span.info_pub').text
        price = book.select_one('div.info_price strong.txt_num').text
        b_list.append({'순위':rank, '책이름':name, '저자|출판사':f'{auth}|{pub}', '가격':price})
df_books = pd.DataFrame(b_list)
df_books.to_csv('ch14_yes24.csv', index=False)
df_books

,순위,책이름,저자|출판사,가격
0,1,트렌드 코리아 2026,"김난도, 전미영, 최지혜, 권정윤, 한다혜 저 외 7명|미래의창","18,000원"
1,2,처음 만나는 양자의 세계,채은미 저|북플레저,"17,100원"
2,3,대형주 추세추종 투자법칙,이종호(전황) 저|사피엔테스,"19,800원"
3,4,위버멘쉬,프리드리히 니체 저|떠오름,"16,020원"
4,5,박곰희 연금 부자 수업,박곰희 저|인플루엔셜,"18,900원"
5,6,주식 투자의 뿌리,박두환 저|애덤스미스,"19,800원"
6,7,절창 切創,구병모 저|문학동네,"16,200원"
7,8,다크 심리학,다크 사이드 프로젝트 저|어센딩,"19,710원"
8,9,단 한 줄만 내 마음에 새긴다고 해도,나민애 저|포레스트북스,"21,420원"
9,10,어른의 행복은 조용하다,태수 저|페이지2북스,"16,020원"
